<a href="https://colab.research.google.com/github/i2mmmmm/kaggle_flood/blob/main/colab/6.%20stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_USERNAME']='mmmmmlee'
os.environ['KAGGLE_KEY']='b4527f98e27813c6dae3fb199096dcb5'

In [ ]:
!kaggle competitions download -c Playground-series-s4e5

 75% 21.0M/28.0M [00:00<00:00, 115MB/s] 
100% 28.0M/28.0M [00:00<00:00, 120MB/s]


In [ ]:
!unzip 'Playground-series-s4e5.zip'

Archive:  Playground-series-s4e5.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

from sklearn.ensemble import VotingRegressor, BaggingRegressor, StackingRegressor
from sklearn.linear_model import BayesianRidge, SGDRegressor, Ridge
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import make_pipeline
import xgboost as xgb
import lightgbm as lgb

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
# train 표준화, 스케일링
scaler = StandardScaler()
X = train.drop(columns=['FloodProbability', 'id'])
y = train['FloodProbability']
X_scaled = scaler.fit_transform(X)

In [ ]:
# feature engineering
train = pd.read_csv('train.csv')

def simplified_getFeats(df):
    num_cols = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement', 'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality', 'Siltation', 'AgriculturalPractices', 'Encroachments', 'IneffectiveDisasterPreparedness', 'DrainageSystems', 'CoastalVulnerability', 'Landslides', 'Watersheds', 'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss', 'InadequatePlanning', 'PoliticalFactors']

    scaler = StandardScaler().fit(df[num_cols])
    df[num_cols] = scaler.transform(df[num_cols])  # 스케일링 먼저 하기

    df['mean'] = df[num_cols].mean(axis=1)
    df['std'] = df[num_cols].std(axis=1)
    df['max'] = df[num_cols].max(axis=1)
    df['min'] = df[num_cols].min(axis=1)

    df['Climate_Risk'] = df['MonsoonIntensity'] * df['ClimateChange']
    df['Infrastructure_Risk'] = df['DamsQuality'] * df['DrainageSystems']
    df['wet_Risk'] = df['WetlandLoss'] * df['Encroachments']

    return df

train_simp = simplified_getFeats(train)

X = train_simp.drop(columns=['FloodProbability', 'id'])
y = train_simp['FloodProbability']

In [ ]:
# 훈련, 검증, 테스트 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=35)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=35)

lgbm_params = {
    'num_leaves': 183,
    'learning_rate': 0.01183688880802108,
    'n_estimators': 577,
    'subsample_for_bin': 165697,
    'min_child_samples': 114,
    'reg_alpha': 2.075080888948164e-06,
    'reg_lambda': 3.838938366471552e-07,
    'colsample_bytree': 0.9634044234652241,
    'subsample': 0.9592138618622019,
    'max_depth': 9
}

cat_params = {
    'n_estimators':8000,
    'random_state':0,
    'learning_rate': 0.011277016304363601,
    'depth': 8,
    'subsample': 0.8675506657380021,
    'min_data_in_leaf': 98,
    'bootstrap_type': 'Bernoulli'
}


# 기본 모델
estimators = [
    ('catboost', CatBoostRegressor(**cat_params)),
    ('lgbm', LGBMRegressor(**lgbm_params))
]

# 메타 모델
stacking_model = StackingRegressor(estimators=estimators, final_estimator=Ridge())

# 모델 학습
stacking_model.fit(X_train, y_train)

# 예측
stacking_predictions = stacking_model.predict(X_test)

# 평가
mae = mean_absolute_error(y_test, stacking_predictions)
print("Test MAE:", mae)

# Test MAE: 0.015838140532246886

# 변수 추가 Test MAE: 0.014793238756400748

# lgbm 추가 Test MAE: 0.01458177811296755

# 선형회귀 삭제 Test MAE: 0.014571303010309587

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
3199:	learn: 0.0185249	total: 9m 28s	remaining: 14m 13s
3200:	learn: 0.0185248	total: 9m 29s	remaining: 14m 13s
3201:	learn: 0.0185247	total: 9m 29s	remaining: 14m 12s
3202:	learn: 0.0185247	total: 9m 29s	remaining: 14m 12s
3203:	learn: 0.0185246	total: 9m 29s	remaining: 14m 12s
3204:	learn: 0.0185245	total: 9m 29s	remaining: 14m 12s
3205:	learn: 0.0185244	total: 9m 29s	remaining: 14m 12s
3206:	learn: 0.0185244	total: 9m 29s	remaining: 14m 11s
3207:	learn: 0.0185243	total: 9m 30s	remaining: 14m 11s
3208:	learn: 0.0185242	total: 9m 30s	remaining: 14m 11s
3209:	learn: 0.0185242	total: 9m 30s	remaining: 14m 11s
3210:	learn: 0.0185241	total: 9m 30s	remaining: 14m 11s
3211:	learn: 0.0185240	total: 9m 30s	remaining: 14m 11s
3212:	learn: 0.0185239	total: 9m 31s	remaining: 14m 10s
3213:	learn: 0.0185238	total: 9m 31s	remaining: 14m 10s
3214:	learn: 0.0185238	total: 9m 31s	remaining: 14m 10s
3215:	learn: 0.0185237	total: 9m 31s	remaining: 14m 10s
3216:	learn:

KeyboardInterrupt: 

In [ ]:
# 기본 모델
estimators = [
    ('catboost', CatBoostRegressor(n_estimators=100, learning_rate=0.1, random_state=35, verbose=0)),
    ('lr', LinearRegression())
]

# 메타 모델
stacking_model = StackingRegressor(estimators=estimators, final_estimator=Ridge())

# 모델 학습
stacking_model.fit(X_train, y_train)

# 예측
stacking_predictions = stacking_model.predict(X_test)

test_simp['FloodProbability'] = stacking_predictions
test_simp[['id','FloodProbability']].to_csv('submission_stacking.csv',index = False)